In [521]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import math

In [522]:
#load the data
products=pd.read_csv('data_set/amazon_baby.csv')
products1=pd.read_csv('data_set/amazon_baby.csv')
products.size, products1.size

(550593, 550593)

In [523]:
products.size

550593

In [524]:
#  removing punctuation, so that words "cake." and "cake!" are counted as the same word
def remove_punctuation(text):
    #text=str(text)
    return text.translate([None,string.punctuation])

## Filling the nan with empty string and remove punctuations

In [526]:
#products[products['review'].isnull()].head()
products=products.fillna({'review':''})
products[products['review'].isnull()].head()
products['review_clean']=products['review'].apply(remove_punctuation)
products.shape

(183531, 4)

In [527]:
products1.shape

(183531, 3)

In [528]:
products.shape

(183531, 4)

## Ignoring all the reviews with rating 3 as they have neutral sentiment

In [530]:
products=products[products['rating']!=3]
products.shape

(166752, 4)

## Rating > 3 is +ve class and < 3 -ve class 

In [531]:
products['sentiment']=products['rating'].apply(lambda rating: +1 if rating > 3 else -1)

In [532]:
products.shape

(166752, 5)

### [NOTE : need products to contain review with 3 ratings also]

In [533]:
indxs=pd.read_json('data_set/module-2-assignment-train-idx.json')
indxs=list(indxs[0].values)

In [534]:
train_data=products.iloc[indxs]

In [535]:
train_data.size

667080

In [536]:
indxs=pd.read_json('data_set/module-2-assignment-test-idx.json')
indxs=list(indxs[0].values)
test_data=products.iloc[indxs]

In [537]:
test_data.columns

Index(['name', 'review', 'rating', 'review_clean', 'sentiment'], dtype='object')

In [538]:
train_data=train_data.fillna({'review':''})
test_data=test_data.fillna({'review':''})

In [539]:
train_data.size,test_data.size

(667080, 166680)

In [540]:
train_data=train_data[train_data['rating']!=3]
train_data['sentiment']=train_data['rating'].apply(lambda x:+1 if x>3 else -1)

In [461]:
train_data.size

667080

## Building the word count vector for each review

In [541]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer(token_pattern=r'\b\w+\b')
train_matrix=vectorizer.fit_transform(train_data['review_clean'])
test_matrix=vectorizer.transform(test_data['review_clean'])

In [542]:
vectorizer.vocabulary_

{'it': 27401,
 'came': 9634,
 'early': 17596,
 'and': 4370,
 'was': 55401,
 'not': 34305,
 'disappointed': 16067,
 'i': 25605,
 'love': 30320,
 'planet': 37855,
 'wise': 56407,
 'bags': 6328,
 'now': 34395,
 'my': 33235,
 'wipe': 56362,
 'holder': 24968,
 'keps': 28258,
 'osocozy': 35582,
 'wipes': 56375,
 'moist': 32466,
 'does': 16669,
 'leak': 29173,
 'highly': 24745,
 'recommend': 41226,
 'very': 54728,
 'soft': 46566,
 'comfortable': 11960,
 'warmer': 55352,
 'than': 50501,
 'looks': 30166,
 'fit': 20583,
 'the': 50557,
 'full': 21855,
 'size': 45673,
 'bed': 6970,
 'perfectly': 37114,
 'would': 56818,
 'to': 51570,
 'anyone': 4656,
 'looking': 30158,
 'for': 21208,
 'this': 51021,
 'type': 52956,
 'of': 34818,
 'quilt': 40406,
 'is': 27323,
 'a': 2581,
 'product': 39419,
 'well': 55785,
 'worth': 56797,
 'purchase': 40032,
 'have': 24252,
 'found': 21403,
 'anything': 4669,
 'else': 18117,
 'like': 29628,
 'positive': 38568,
 'ingenious': 26497,
 'approach': 4852,
 'losing': 3026

## Training a classifier with logistic regression

In [543]:
from sklearn.linear_model import LogisticRegression
sentiment_model=LogisticRegression()
sentiment_model.fit(train_matrix,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### Quiz question: How many weights are >= 0? **

In [546]:
coefficients = list(sentiment_model.coef_[0])
positive_weight=[]
for coef in coefficients:
    if coef>=0:
        positive_weight.append(coef)
len(positive_weight)

39487

In [471]:
sample_test_data=test_data[10:13]
sample_test_data.head()

,name,review,rating,review_clean,sentiment
59,Our Baby Girl Memory Book,Absolutely love it and all of the Scripture in...,5,Absolutely love it and all of the Scripture in...,1
71,Wall Decor Removable Decal Sticker - Colorful ...,Would not purchase again or recommend. The dec...,2,Would not purchase again or recommend. The dec...,-1
91,New Style Trailing Cherry Blossom Tree Decal R...,Was so excited to get this product for my baby...,1,Was so excited to get this product for my baby...,-1


In [472]:
sample_test_data.iloc[0]['review_clean']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

## Use of decision function
Recall from the lecture that the score (sometimes called margin) for the logistic regression model is defined as:
<br>
$score_{i}=w^{⊺}h(x_{i})$<br>
where $h(x_{i})$ represents the features for data point i. We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

In [473]:
sample_test_matrix=vectorizer.transform(sample_test_data['review_clean'])
scores=sentiment_model.decision_function(sample_test_matrix)
prediction_score=[]
for score in scores:
    if score>0:
        prediction_score.append(+1)
    elif score<0:
        prediction_score.append(-1)
prediction_score,scores

([1, -1, -1], array([  4.96982261,  -3.0026716 , -10.60546377]))

In [474]:
sentiment_model.predict(sample_test_matrix)

array([ 1, -1, -1], dtype=int64)

In [475]:
def prediction_scores(scores):
    prediction_score=[]
    for score in scores:
        if score>0:
            prediction_score.append(+1)
        elif score<0:
            prediction_score.append(-1)
    return prediction_score

In [476]:
def prediction_probabs(scores):
    probs=[]
    for score in scores:
        p=1/(1+math.exp(-score))
        probs.append(p)
    return probs

In [477]:
probs=prediction_probabs(scores)
probs

[0.9931035120694313, 0.047305325179694054, 2.4779631617439094e-05]

In [478]:
sentiment_model.predict_proba(sample_test_matrix)[0][1]

0.9931035120694313

### Quiz question: Of the three data points in sample_test_data, which one (first, second, or third) has the lowest probability of being classified as a positive review?

In [479]:
min(probs)

2.4779631617439094e-05

In [480]:
test_matrix=vectorizer.transform(test_data['review_clean'])

In [481]:
proba_20=sentiment_model.predict_proba(test_matrix)

In [482]:
prob=[]
for p in proba_20:
    prob.append(p[1])
prob.sort()

In [483]:
prob[-21:-1]

[0.9999999999999998,
 0.9999999999999998,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [484]:
for p in prob[-21:-1]:
    if p

SyntaxError: invalid syntax (<ipython-input-484-581d9d237866>, line 2)

In [485]:
len(proba_20)

33336

In [234]:
p=1.0
np.where(p == proba_20)

(array([ 4348, 10113, 17286, 19928, 23127, 24710, 26677, 28802, 31589,
        32192, 32439], dtype=int64),
 array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64))

In [486]:
probs=[]
for p in proba_20:
    probs.append(p[1])

### Quiz Question: Which of the following products are represented in the 20 most positive reviews?

In [487]:
indices=np.argsort(probs)
indices[-21:-1]

array([ 2570, 27048, 15732, 32782, 21203, 30634, 24286, 21531, 18112,
       30076, 11923,  9125,  9555, 26830, 17558, 33060,  4140, 16502,
       24899, 14482], dtype=int64)

### Quiz Question: Which of the following products are represented in the 20 most negative reviews?

In [488]:
indx_min=np.argsort(probs)
indx_min[0:20]

array([28184, 21700, 17069,  2931,  8818, 13939,  1810,  9655, 20594,
       13751,  1942, 14711, 15062, 31928, 17222,   205, 28120, 27231,
       30373, 10814], dtype=int64)

In [489]:
test_data['sentiment']=test_data['rating'].apply(lambda rating: +1 if rating >3 else -1)

In [490]:
predictions=sentiment_model.predict(test_matrix)

In [491]:
corrects = predictions == test_data['sentiment']
np.sum(corrects)

31075

In [492]:
def accuracy_value(predictions,output):
    total=len(output)
    corrects=predictions == output
    correctly_classified=np.sum(corrects)
    return (correctly_classified/total)

In [493]:
accuracy_sentiment_model=accuracy_value(predictions,test_data['sentiment'])
accuracy_sentiment_model

0.9321754259659227

### Quiz Question: What is the accuracy of the sentiment_model on the test_data? Round your answer to 2 decimal places **

In [494]:
from sklearn.metrics import accuracy_score
score=accuracy_score(test_data['sentiment'],predictions)
score

0.9321754259659227

## Learn another classifier with fewer words

In [495]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [496]:
vectorizer_word_subset=CountVectorizer(vocabulary=significant_words) #Limits to 20 words

In [497]:
train_matrix_words=vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_words=vectorizer_word_subset.transform(test_data['review_clean'])

In [498]:
simple_model=LogisticRegression()
simple_model.fit(train_matrix_words,train_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [499]:
coeffs=simple_model.coef_
d = {'words':['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return'],'coefficients':[ 1.35268316,  0.91986614,  1.17294417,  0.11500178,  0.49734749,
         1.51080771,  1.7519224 ,  0.47764534,  0.18802123,  0.08941532,
        -1.64278681, -0.17504869, -0.51982782, -1.99557715, -2.36196483,
        -0.63499832, -0.31321812, -0.86247905, -0.33653428, -2.01676118]}
df=pd.DataFrame(data=d)
df.head()

,words,coefficients
0,love,1.352683
1,great,0.919866
2,easy,1.172944
3,old,0.115002
4,little,0.497347


In [500]:
df.sort_values(by=['coefficients'],ascending=False)

,words,coefficients
6,loves,1.751922
5,perfect,1.510808
0,love,1.352683
2,easy,1.172944
1,great,0.919866
4,little,0.497347
7,well,0.477645
8,able,0.188021
3,old,0.115002
9,car,0.089415


### Quiz Question: Consider the coefficients of simple_model. How many of the 20 coefficients (corresponding to the 20 significant_words) are positive for the simple_model?

In [501]:
np.sum(df['coefficients'].values > 0)

10

### Quiz Question: Are the positive words in the simple_model also positive words in the sentiment_model?

In [502]:
sentiment_model.coef_

array([[-5.44520531e-01, -6.11804000e-01, -5.82232457e-02, ...,
         1.72839802e-02,  3.15923578e-03, -5.97656429e-05]])

In [503]:
idx_sentiment=[]
for word in significant_words:
    idx_sentiment.append(vectorizer.vocabulary_.get(word))
idx_sentiment
for idx in idx_sentiment:
    print(sentiment_model.coef_[0][idx])

1.541260504650111
1.2189277951693092
1.3506620171965975
0.04066916805130735
0.6260677633409618
1.8325795680496908
1.4854892864244482
0.5733529390273953
0.4044147795868985
0.09763585439417836
-1.3462249107842401
-0.2932268200959007
-0.4573907952384159
-1.9295488676990367
-2.1824354103997066
-0.45106180157238007
-0.1996810604200818
-0.777915629397156
-0.2991809158146148
-1.6231969095695902


### Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TRAINING set?

In [504]:
predict_sentiment=sentiment_model.predict(train_matrix)
accuracy_score(predict_sentiment,train_data['sentiment'])

0.9624183006535948

In [505]:
predict_simple=simple_model.predict(train_matrix_words)
accuracy_score(predict_simple,train_data['sentiment'])

0.8672273190621814

### Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TEST set? **

In [506]:
predict_sentiment=sentiment_model.predict(test_matrix)
accuracy_score(predict_sentiment,test_data['sentiment'])

0.9321754259659227

In [507]:
predict_simple=simple_model.predict(test_matrix_words)
accuracy_score(predict_simple,test_data['sentiment'])

0.8702003839692825

In [508]:
train_data.groupby('sentiment').count()

,name,review,rating,review_clean
sentiment,,,,
-1,21227,21252,21252,21252
1,111947,112164,112164,112164


## Majority class classifier

In [509]:
predict_majority=np.repeat(1,test_data['sentiment'].size)

In [510]:
accuracy_score(predict_majority,test_data['sentiment'])

0.8427825773938085

In [511]:
for i in indices[-21:-1]:
    print(test_data.iloc[i])
    print("______________________")

name                Stork Craft Beatrice Combo Tower Chest, White
review          I bought the tower despite the bad reviews and...
rating                                                          5
review_clean    I bought the tower despite the bad reviews and...
sentiment                                                       1
Name: 14008, dtype: object
______________________
name            The Original CJ's BuTTer (All Natural Mango Su...
review          I'm going to try to review the scents I've tri...
rating                                                          4
review_clean    I'm going to try to review the scents I've tri...
sentiment                                                       1
Name: 149002, dtype: object
______________________
name              Baby Einstein Around The World Discovery Center
review          I am so HAPPY I brought this item for my 7 mon...
rating                                                          5
review_clean    I am so HAPPY I brought t

In [512]:
indx_min[0:20]

array([28184, 21700, 17069,  2931,  8818, 13939,  1810,  9655, 20594,
       13751,  1942, 14711, 15062, 31928, 17222,   205, 28120, 27231,
       30373, 10814], dtype=int64)

In [513]:
for i in indx_min[0:20]:
    print(test_data.iloc[i])
    print("______________________")

name            VTech Communications Safe &amp; Sounds Full Co...
review          This is my second video monitoring system, the...
rating                                                          1
review_clean    This is my second video monitoring system, the...
sentiment                                                      -1
Name: 155287, dtype: object
______________________
name            Levana Safe N'See Digital Video Baby Monitor w...
review          This is the first review I have ever written o...
rating                                                          1
review_clean    This is the first review I have ever written o...
sentiment                                                      -1
Name: 120209, dtype: object
______________________
name            The First Years True Choice P400 Premium Digit...
review          Note: we never installed batteries in these un...
rating                                                          1
review_clean    Note: we never installed